In [7]:
from datasets import load_dataset
import os
from PIL import Image
from tqdm import tqdm

# Load the dataset
dataset = load_dataset('aquib1011/maize-leaf-disease', 'default')

# Define label to class name mapping
label_to_class_name = {
    0: 'Blight',
    1: 'Common_Rust',
    2: 'Gray_Leaf_Spot',
    3: 'Healthy'
}

# Create directories if they don't exist
os.makedirs('data1/train', exist_ok=True)
os.makedirs('data1/test', exist_ok=True)

def save_images(dataset_split, split_name):
    for i, item in enumerate(tqdm(dataset_split)):
        image = item['image']
        label = item['label']
        class_name = label_to_class_name[label]  # Map label to class name
        # Create subdirectory for the class name if it doesn't exist
        class_dir = os.path.join(f'data1/{split_name}', class_name)
        os.makedirs(class_dir, exist_ok=True)
        # Convert image to RGB if it's in RGBA mode
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        # Save the image
        image.save(os.path.join(class_dir, f"{class_name}_{i}.jpg"))

# Save train and test images
save_images(dataset['train'], 'train')
save_images(dataset['test'], 'test')


100%|██████████| 838/838 [00:06<00:00, 125.39it/s]


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for train and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory for train and test sets
train_generator = train_datagen.flow_from_directory(
    'data1/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    'data1/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)


Found 3350 images belonging to 4 classes.
Found 838 images belonging to 4 classes.


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Adjust '4' to match the number of classes you have
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)


Epoch 1/30
105/105 [==============================] - 86s 808ms/step - loss: 0.7593 - accuracy: 0.6875 - val_loss: 0.6643 - val_accuracy: 0.7363
Epoch 2/30
105/105 [==============================] - 83s 786ms/step - loss: 0.4750 - accuracy: 0.8122 - val_loss: 0.4156 - val_accuracy: 0.8186
Epoch 3/30
105/105 [==============================] - 80s 761ms/step - loss: 0.4429 - accuracy: 0.8149 - val_loss: 0.4192 - val_accuracy: 0.8174
Epoch 4/30
105/105 [==============================] - 83s 785ms/step - loss: 0.4388 - accuracy: 0.8185 - val_loss: 0.4038 - val_accuracy: 0.8210
Epoch 5/30
105/105 [==============================] - 78s 739ms/step - loss: 0.3834 - accuracy: 0.8319 - val_loss: 0.3719 - val_accuracy: 0.8413
Epoch 6/30
105/105 [==============================] - 80s 764ms/step - loss: 0.3650 - accuracy: 0.8352 - val_loss: 0.3376 - val_accuracy: 0.8389
Epoch 7/30
105/105 [==============================] - 79s 748ms/step - loss: 0.3930 - accuracy: 0.8275 - val_loss: 0.3588 - val_ac

In [11]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_acc:.4f}')


27/27 [==============================] - 3s 120ms/step - loss: 0.2716 - accuracy: 0.9033
Test accuracy: 0.9033


In [12]:
model.save("Mazie_leaf_disease_detector.h5")

c:\Users\USER\anaconda3\envs\maizeDisease\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# to test the model
from tensorflow.keras.models import load_model

model = load_model("./model/Mazie_leaf_disease_detector.h5")

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0  # Normalize the image


In [19]:
# Path to your image
image_path = './data1/train/Common_Rust/Common_Rust_1010.jpg'

# Preprocess the image
processed_img = preprocess_image(image_path)

# Make predictions
predictions = model.predict(processed_img)

predicted_class = np.argmax(predictions[0])

# Map the predicted class index to your class labels
class_mapping = {
    0: 'Blight',
    1: 'Common_Rust',
    2: 'Gray_Leaf_Spot',
    3: 'Healthy'
}

predicted_label = class_mapping[predicted_class]
print(f'Predicted class: {predicted_label}')


1/1 [==============================] - 0s 42ms/step
Predicted class: Common_Rust


CLASSIFICATION REPORT

In [27]:
!pip install scikit-learn


  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   ---------- ---------------------------- 81.9/301.8 kB 327.3 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/301.8 kB 385.0 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/301.8 kB 405.9 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/301.8 kB 405.9 kB/s eta 0:00:01
   --------------------- ---------------- 174.1/301.8 kB 403.5 kB/s eta 0:00:01
   --------------------------

In [30]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import classification_report

# Load your saved model (replace 'detector.h5' with your actual model file)
model = load_model('./model/Mazie_leaf_disease_detector.h5')

# Function to preprocess an image
def preprocess_image(image_path):
    try:
        img = image.load_img(image_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return img_array / 255.0  # Normalize the image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None  # Handle gracefully in your application

# List of test image paths (replace with your actual test image paths)
test_image_paths = [
    './data1/train/Blight/Blight_102.jpg',
    './data1/train/Healthy/Healthy_1009.jpg',
]

predictions = []

# Process each test image and make predictions
for image_path in test_image_paths:
    processed_img = preprocess_image(image_path)
    if processed_img is not None:
        prediction = model.predict(processed_img)
        predicted_class = np.argmax(prediction[0])
        predictions.append(predicted_class)
    else:
        print(f"Skipping prediction for {image_path} due to processing error.")

# Assuming you have ground truth labels for your test images (replace with actual labels)
true_labels = [0, 3] 

# Generate and print classification report
report = classification_report(true_labels, predictions)
print(report)


1/1 [==============================] - 0s 115ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

